In [ ]:
import numpy as np
from nbodykit.lab import cosmology
import os
import sys

module_path = os.path.abspath(os.path.join("../lib"))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pk_tools import pk_tools
import pk_fit_selection as fit

In [ ]:
datapath = "../data/BOSS_DR12_NGC_z1/"
datafile = datapath + "ps1D_BOSS_DR12_NGC_z1_COMPnbar_TSC_700_700_700_400_renorm.dat"
W = pk_tools.read_matrix(datapath + "W_BOSS_DR12_NGC_z1_V6C_1_1_1_1_1_10_200_2000_averaged_v1.matrix")
M = pk_tools.read_matrix(datapath + "M_BOSS_DR12_NGC_z1_V6C_1_1_1_1_1_2000_1200.matrix")

pk_data_dict = pk_tools.read_power(datafile, combine_bins=10)
kbins, _ = pk_tools.dict_to_vec(pk_data_dict)
Nmodes = np.tile(pk_data_dict['Nmodes'], 5)

pk_model_dict = pk_tools.read_power(datafile, combine_bins=1)
krange = pk_model_dict["k"]

z = 0.38
cosmo = cosmology.Planck15
pk_matter = cosmology.LinearPower(cosmo, redshift=z, transfer="EisensteinHu")
pk_matter_k = pk_matter(krange)

b = 2
f = 0.7

P0 = (b**2 + 2/3*b*f + 1/5*f**2) * pk_matter_k
P2 = (4/3*b*f + 4/7*f**2) * pk_matter_k
P4 = 8/35*f**2*pk_matter_k

P = np.concatenate((P0, P2, P4))
P_conv = W@M@P

fit_selection = fit.get_fit_selection(kbins, kmin=0.01, kmax=0.1, pole_selection=[True, False, True, False, False])
fit_P_conv = P_conv[fit_selection]
fit_Nmodes = Nmodes[fit_selection]

T = np.diag(2*fit_P_conv**2 / fit_Nmodes)

p = np.sum(fit_selection)
#np.savetxt(f"../output/BOSS_DR12_NGC_z1/T_{p}_{p}.matrix", T)